### import packages

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

### global vars

In [2]:
start_date = 19570101  # start date
end_date = 20201231  # end date
ch_data_path = "./dataset/GKX_20201231.csv" # characteristic data path
mp_data_path = "./dataset/PredictorData2023Monthly.csv" # macro predictors data path
full_clean_data_path = "./dataset/full_dataset.csv" # clean data path
top_clean_data_path = "./dataset/top_dataset.csv" # clean data path
bottom_clean_data_path = "./dataset/bottom_dataset.csv" # clean data path

### load characteristic data and clean

In [3]:
ch_data = pd.read_csv(ch_data_path)

# select data according to date and format date
ch_data = ch_data.loc[(ch_data['DATE'] >= start_date) & (ch_data['DATE'] <= end_date)].reset_index(drop=True)
ch_data['DATE'] = pd.to_datetime(ch_data['DATE'], format='%Y%m%d') + pd.offsets.MonthEnd(0) # for filling null data

# extract characteristic
exclude_columns = ['permno', 'DATE', 'sic2', 'RET', 'prc','SHROUT','mve0']
characteristics = ch_data.columns.difference(exclude_columns).tolist()

# fill missing characteristic
for ch in characteristics:
    ch_data[ch] = ch_data.groupby('DATE')[ch].transform(lambda x: x.fillna(x.median()))
for ch in characteristics:
    ch_data[ch] = ch_data[ch].fillna(0)

# drop unused columns
ch_data.drop(['sic2', 'permno', 'prc','SHROUT','mve0'], axis=1, inplace=True)

### load macroeconomic predictors data and clean

In [4]:
mp_data = pd.read_csv(mp_data_path)

# select data according to date and format date
mp_data = mp_data[(mp_data['yyyymm']>=start_date/100)&(mp_data['yyyymm']<=end_date//100)].reset_index(drop=True)
mp_data['DATE'] = pd.to_datetime(mp_data['yyyymm'], format='%Y%m')  + pd.offsets.MonthEnd(0)

# calculate additional variables based on mp_data
mp_data['Index'] = mp_data['Index'].str.replace(',','').astype('float')
mp_data['d/p'] = mp_data['D12']/mp_data['Index']
mp_data['e/p'] = mp_data['E12']/mp_data['Index']
mp_data['tms'] = mp_data['lty'] - mp_data['tbl']
mp_data['dfy'] = mp_data['BAA'] - mp_data['AAA']

# drop unused columns
mp_data.drop(['Index', 'D12', 'E12', 'AAA', 'yyyymm','BAA', 'lty', 'Rfree', 'infl', 'ltr', 'corpr', 'csp', 'CRSP_SPvw', 'CRSP_SPvwx',], axis=1, inplace=True)

### construct final data

In [5]:
# merge ch and mp data
data = pd.merge(ch_data, mp_data, how='left', on='DATE').reset_index(drop=True)


# construct covariates
# ch_data.drop(['DATE'], axis=1, inplace=True)
# mp_data.drop(['DATE'], axis=1, inplace=True)
# chs = list(set(ch_data.columns))
# mps = list(set(mp_data.columns))
# covariates = pd.DataFrame()
# for ch in chs:
#     for mp in mps:
#         covariates[ch+'*'+mp] = ch_data[ch] * mp_data[mp]
# data = pd.concat([data, covariates], axis=1)

# for gc
del ([ch_data, mp_data])

# store data
data.to_csv(full_clean_data_path, index=None)

data_top = data.sort_values('mvel1',ascending=False).groupby('DATE').head(1000).reset_index(drop=True) # top 1000
data_top.to_csv(top_clean_data_path, index=None)
del data_top

data_bot = data.sort_values('mvel1',ascending=False).groupby('DATE').tail(1000).reset_index(drop=True) # bottom 1000
data_bot.to_csv(top_clean_data_path, index=None)
del data_bot


KeyboardInterrupt: 